<a href="https://colab.research.google.com/github/tawfig2020/AIBISz/blob/master/Predicating_The_Best_Time_For_Scheduling_E_Mail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install schedule

In [4]:
import pandas as pd
import numpy as np
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scipy.stats import ttest_ind
import schedule
import time
from requests import post, patch


**# Example user data**

In [5]:
data = {
    'user_id': range(1, 11),
    'age': [25, 35, 45, 55, 65, 30, 40, 50, 60, 70],
    'purchase_count': [5, 3, 10, 2, 8, 4, 6, 9, 1, 7],
    'total_spend': [500, 300, 1000, 200, 800, 400, 600, 900, 100, 700],
    'email': [
        'user1@example.com', 'user2@example.com', 'user3@example.com',
        'user4@example.com', 'user5@example.com', 'user6@example.com',
        'user7@example.com', 'user8@example.com', 'user9@example.com',
        'user10@example.com'
    ]
}
df = pd.DataFrame(data)


**# Normalize user features and segment using KMeans**

In [6]:
scaler = StandardScaler()
features = ['age', 'purchase_count', 'total_spend']
df_scaled = scaler.fit_transform(df[features])
kmeans = KMeans(n_clusters=3, random_state=42)
df['segment'] = kmeans.fit_predict(df_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


**# Personalize content based on spending**

In [7]:
df['personalized_message'] = np.where(df['total_spend'] > 500,
                                      "Check out our premium offers!",
                                      "Discover great value deals!")

**# Example historical engagement data (mock data)**

In [15]:
historical_data = {
    'user_id': range(1, 11),
    'send_time': [8, 10, 14, 18, 20, 9, 11, 15, 19, 21],
    'opened': [1, 0, 1, 1, 0, 1, 0, 1, 1, 0],
    'clicked': [0, 0, 1, 0, 0, 0, 1, 0, 1, 0]
    }
df_historical = pd.DataFrame(historical_data)
engagement_df = pd.DataFrame(historical_data)
df = df.merge(engagement_df, on='user_id')

**# Prepare data for training the model**

In [16]:
X = df[['age', 'purchase_count', 'total_spend', 'send_time']]
y = df['opened']

**# Train-test split**

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

**# Build a simple model for predicting engagement**

In [18]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

**# Train the model**

In [19]:
model.fit(X_train, y_train, epochs=20, batch_size=2, verbose=1)

Epoch 1/20
4/4 [==============================] - 1s 7ms/step - loss: 29.6305 - accuracy: 0.2857
Epoch 2/20
4/4 [==============================] - 0s 5ms/step - loss: 14.6465 - accuracy: 0.4286
Epoch 3/20
4/4 [==============================] - 0s 5ms/step - loss: 18.8338 - accuracy: 0.7143
Epoch 4/20
4/4 [==============================] - 0s 5ms/step - loss: 11.9417 - accuracy: 0.7143
Epoch 5/20
4/4 [==============================] - 0s 5ms/step - loss: 9.4869 - accuracy: 0.5714 
Epoch 6/20
4/4 [==============================] - 0s 5ms/step - loss: 15.5443 - accuracy: 0.2857
Epoch 7/20
4/4 [==============================] - 0s 6ms/step - loss: 37.0202 - accuracy: 0.1429
Epoch 8/20
4/4 [==============================] - 0s 5ms/step - loss: 6.5124 - accuracy: 0.5714
Epoch 9/20
4/4 [==============================] - 0s 5ms/step - loss: 17.4223 - accuracy: 0.5714
Epoch 10/20
4/4 [==============================] - 0s 7ms/step - loss: 24.5305 - accuracy: 0.2857
Epoch 11/20
4/4 [=============

**# Evaluate the model**

In [20]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)
print(classification_report(y_test, y_pred))

1/1 [==============================] - 0s 168ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**# Predict the optimal send time for each user**

In [29]:
if 'optimal_send_time' not in df.columns:
    # Add the column with appropriate values
    optimal_send_times = []
    for index, row in df.iterrows():
        user_features = np.array([[row['age'], row['purchase_count'], row['total_spend'], t] for t in range(8, 22)])
        send_time_predictions = model.predict(user_features)
        optimal_send_time = np.argmax(send_time_predictions) + 8
        optimal_send_times.append(optimal_send_time)

    df['optimal_send_time'] = optimal_send_times
else:
    raise KeyError("The 'optimal_send_time' key does not exist in the DataFrame.")


1/1 [==============================] - 0s 24ms/step


**# Function to send an email**

In [30]:
def send_email(recipient, subject, body):
    message = MIMEMultipart()
    message['From'] = 'your_email@example.com'
    message['To'] = recipient
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.example.com', 587)
        server.starttls()
        server.login('your_email@example.com', 'your_password')
        server.sendmail('your_email@example.com', recipient, message.as_string())
        server.quit()
        print(f"Email sent to {recipient}")
    except Exception as e:
        print(f"Failed to send email to {recipient}: {str(e)}")

**# Function to schedule email sending at the optimal time**

In [31]:
def schedule_emails():
    for _, row in df.iterrows():
        recipient = row['email']
        subject = 'Exclusive Offer Just for You!'
        body = row['personalized_message']
        send_time = row['optimal_send_time']

        def job(recipient=recipient, subject=subject, body=body, send_time=send_time):
            send_email(recipient, subject, body)

        schedule.every().day.at(f"{send_time:02d}:00").do(job)

**# Run the email scheduler**

In [33]:
schedule_emails()